In [1]:
import gzip
import re

In [2]:
with gzip.open("../data/poczatki_wikipediowe.txt.gz", "rt") as f:
    file_content = f.read()

In [3]:
d = {}
counter = 0
for part in file_content.removeprefix("### ").split("\n### "):
    splited = part.strip().split("\n", maxsplit=1)
    if len(splited) == 2:
        title, content = splited
        if title in d:
            counter += 1
        d[title] = content

In [4]:
skrot_pattern = re.compile(r"'''([\w\s]+)''' \(w skrócie '''([\w\s]+)'''\)")
re.findall(skrot_pattern, file_content)[:5]

[('Quake C', 'QC'),
 ('Senatus Populusque Romanus', 'SPQR'),
 ('General Motors Corporation', 'GM'),
 ('Państwowe Gospodarstwo Rolne', 'PGR'),
 ('Generalny Inspektor Ochrony Danych Osobowych', 'GIODO')]

In [5]:
base_pattern = re.compile(r"'''([\w\s]+)'''(?:(?:,|(?:,? lub)) '''([\w\s]+)''')*")
definition_pattern = re.compile(r"'''(?<!,)([\w\s]+)'''")

def extract_synonyms(title, content):
    if content.startswith("* "):
        # enumeration
        return set()
    elif m := base_pattern.match(content):
        synonyms = set(definition_pattern.findall(m.group()))
        for skrot_set in map(set, skrot_pattern.findall(content)):
            if not skrot_set.isdisjoint(synonyms):
                synonyms.update(skrot_set)
        return synonyms
    else:
        return set()

In [6]:
m = base_pattern.match("'''Wizygoci''', '''Goci zachodni''' lub '''Terwingowie''' (dosł. leśni ludzie ) – lud germańs")
print(definition_pattern.findall(m.group()))
print(m.group())

['Wizygoci', 'Goci zachodni', 'Terwingowie']
'''Wizygoci''', '''Goci zachodni''' lub '''Terwingowie'''


In [7]:
translation_pattern = r" *\(\w+\. ''[\w\s]+''\)"

In [8]:
counter = 0

for k, v in d.items():
    synonyms = extract_synonyms(k, v)
    if (extract_synonyms(k, re.sub(translation_pattern, "", v))).difference(synonyms):
        print(v)
        counter += 1
        if counter > 5:
            break

'''Emotikon''' lub '''emotikona''' (ang. ''emoticon''), '''uśmieszek''' (''smile'', ''smiley''), '''buźka''' – złożony ze znaków tekstowych (ASCII) wyraz nastroju, używany przez użytkowników Internetu. Najczęściej przedstawia symboliczny ludzki grymas twarzy, obrócony o 90° w kierunku przeciwnym do wskazówek zegara, w licznych wariantach. Niektóre programy jak np.
'''Goldwasser''' (niem. ''złota woda''), '''gdańska wódka''' – mocny (minimum 40 proc. alkoholu) likier ziołowo-korzenny (według pierwotnej receptury oparty na mieszaninie 20 ziół) wytwarzany według pomysłu holenderskiego imigranta z XVI w. w Gdańsku. Charakterystyczną cechą tego trunku są drobne płatki 23-karatowego złota, które są zawieszone w tym alkoholu.
'''Przysadka mózgowa (łac. ''hypophysis'')''' – gruczoł dokrewny o masie 0,7 g, którego funkcją jest wytwarzanie i wydzielanie hormonów. Jest ona zlokalizowana wewnątrz czaszki w okolicy kostnego zagłębienia nazywanego siodłem tureckim (łac. ''sella turcica''). Przysadka

In [9]:
def extract_without_translations(title, content):
    return extract_synonyms(title, re.sub(translation_pattern, "", content))

In [10]:
m = base_pattern.match("'''Emotikon''' lub '''emotikona''' (ang. ''emoticon''), '''uśmieszek''' (''smile'', ''smiley''), '''buźka''' – złożony ze zn")
print(definition_pattern.findall(m.group()))
print(m.group())

['Emotikon', 'emotikona']
'''Emotikon''' lub '''emotikona'''


In [11]:
with open("out", "wt") as f:
    for k, v in d.items():
        if v.startswith("* "):
            # enumeration
            continue

        synonyms = extract_without_translations(k, v)
        if len(synonyms) > 1:
            f.write(" # ".join(synonyms) + "\n")

In [75]:
useful = [
    "Wizygoci", "Emotikon", "Reakcja termojądrowa",
    "Komórka blastyczna", "Kasa Rolniczego Ubezpieczenia Społecznego",
    "Przysadka mózgowa", "ScTP", "Komitet Narodowy Polski i Ziem Zabranych",
    "Adresarka", "Ahmose II", "Estuary English"
]
for title in useful:
    print(title, d[title], "Extracted: " + str(extract_without_translations(title, d[title])), sep="\n", end="\n\n")

Wizygoci
'''Wizygoci''', '''Goci zachodni''' lub '''Terwingowie''' (dosł. leśni ludzie ) – lud germański, odłam Gotów. W IV wieku przyjęli arianizm (dzięki Biblii przetłumaczonej przez Wulfilę na język gocki). = = Wczesne dzieje = = Pierwsze przekazy traktujące Wizygotów jako osobny lud odwołują się do roku 268, w którym najechali oni cesarstwo rzymskie i opanowali Półwysep
Extracted: {'Wizygoci', 'Terwingowie', 'Goci zachodni'}

Emotikon
'''Emotikon''' lub '''emotikona''' (ang. ''emoticon''), '''uśmieszek''' (''smile'', ''smiley''), '''buźka''' – złożony ze znaków tekstowych (ASCII) wyraz nastroju, używany przez użytkowników Internetu. Najczęściej przedstawia symboliczny ludzki grymas twarzy, obrócony o 90° w kierunku przeciwnym do wskazówek zegara, w licznych wariantach. Niektóre programy jak np.
Extracted: {'uśmieszek', 'Emotikon', 'emotikona'}

Reakcja termojądrowa
'''Reakcja termojądrowa''', '''synteza jądrowa''' lub '''fuzja jądrowa''' – '''Reakcja termojądrowa''', '''synteza jąd

In [61]:
useful = [
    "Zabytki Głogowa", "Powiat wasylkowski", "Psianka podłużna",
    "Salala"
]
for title in useful:
    print(title, d[title], "Extracted: " + str(extract_without_translations(title, d[title])), sep="\n", end="\n\n")

Zabytki Głogowa
'''Głogów''' jest jednym z najstarszych '''Głogów''' jest jednym z najstarszych miast w Polsce. Pierwotnie znajdował się tu gród plemienia Dziadoszan, później na terenie Ostrowa Tumskiego Mieszko I zbudował nowy gród. Do historii przeszła bohaterska obrona grodu przed Niemcami w roku 1109. Spalony w roku 1157 przez Fryderyka I Barbarossę i znów odbudowany, stał się
Extracted: {'Głogów'}

Powiat wasylkowski
'''Powiat''' lub '''ujezd wasylkowski''' dawny powiat, położony w środkowej części guberni kijowskiej. Siedzibą władz było miasto Wasylków. Opis powiatu wasylkowskiego pod względem historycznym, obyczajowym i statystycznym wydał Edward Rulikowski (Warsz., 1853). = = Linki zewnętrzne = = * *
Extracted: {'ujezd wasylkowski', 'Powiat'}

Psianka podłużna
'''Psianka podłużna''' (''Solanum melongena'') to prawidłowa nazwa botaniczna gatunku roślin z rodziny psiankowatych. Roślina ta jednak znana jest głównie pod zwyczajowymi nazwami jako oberżyna, bakłażan, ale ma także inn